In [1]:
# Test conntection
# Should work when running from local inference. 
# On Kaggle, it requires phone verification

import socket, warnings

try: 
    socket.setdefaulttimeout(1)
    socket.socket(socket.AF_INET, socket.SOCK_STREAM).connect(('1.1.1.1', 53))
    print('Connection OK')
except socket.error as ex: raise Exception("No internet connection")

In [ ]:
from duckduckgo_search import ddg_images
from fastcore.all import *

def search_images(term, max_images=200): return L(ddg_images(term, max_results=max_images)).itemgot('image')